<a href="https://colab.research.google.com/github/PUC-RecSys-Class/RecSysPUC-2020/blob/master/practicos/pyRecLab_SlopeOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


Slope One o pendiente uno es una forma simple de filtrado colaborativo basa en items y ratings de usuario. La ventaja que proponen los autores es que es simple de implementer y es bastante eficiente.

El filtrado colaborativo aputa a predecir el rating que un individuo le va a otorgar a un elemento, basandode en un conjunto de usuarios que ya han rankiado o evaluado a estos elementos.

En vez de usar una regresion, utilizan un parametro libre que lo que hace es represetar la desviacion promedio entre ratings de dos elementos.

# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  completa tu nombre aquí :D

## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**

a) Es un filtrado colaborativo ya que requiere evaluaciones de otros usuarios para poder predecir la evaluacion del usuario activo a un item en especifico.

b) Este tipo de filtrado esta basado en usuarios.

c) Slope One recibe las evaluaciones que los usuarios le han dado a ciertos elementos. En este caso, necesitamos el rating, el usuario y el elemento. Entonces para predecir la evaluacion de un usuario, tomamos usuarios que compartan ratings con el usuario activo. Luego, hacemos una diferencia de las calificaciones para luego ponderarlas por la cantidad de elementos que comparten su evaluacion. Luego, finalmente con estas desviaciones ponderadas, la dividimos en la cantidad de muestras (elementos) y obtenemos un rating.

d) Depende si esta evaluacion comparte con el usuario activo. En este caso, debiesemos volver a hacer los calculos con la nueva informacion obtenida.

e) El problema planteado es el de "inicio frio", donde al haber pocos ratings puede costar encontrar ratings que compartan un par de usuarios para ejecutar el algoritmo y obtener predicciones.




# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [1]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    274      0 --:--:--  0:00:01 --:--:--   274
100 1546k  100 1546k    0     0   895k      0  0:00:01  0:00:01 --:--:--  895k


In [2]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    451      0 --:--:-- --:--:-- --:--:--   451
100  385k  100  385k    0     0   357k      0  0:00:01  0:00:01 --:--:--  357k


In [3]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    717      0 --:--:-- --:--:-- --:--:--   717
100  230k  100  230k    0     0   299k      0 --:--:-- --:--:-- --:--:--  299k


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [4]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 234 kB 28.7 MB/s 


## Paso 3:

Hacemos los imports necesarios para este práctico.

In [5]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [6]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [7]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [8]:
# Y enntrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:**

## Paso 6:

Llego la hora de predecir el rating.

In [10]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores

## Lo cambiaremos al usuario 45
mySlopeOne.predict("45", "37")

2.6857142448425293

In [11]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("45", "37")

In [12]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 45]

,userid,itemid,rating,timestamp
3727,45,1,5,881013176
3728,45,7,3,881008080
3729,45,15,4,881012184
3730,45,21,3,881014193
3731,45,24,3,881014550
3732,45,25,4,881014015
3733,45,100,5,881010742
3734,45,108,4,881014620
3735,45,109,5,881012356
3736,45,111,4,881011550


In [13]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?  

**Respuesta:** ¿ Que resultados ? No comprendí bien el objetivo de esta pregunta. Sin embargo, el rating predicho de 2.68 para el usuario 45 se evidencia en los ratings que tiene la pelicula, que rondan entre ese valor.

In [20]:
prediction_id9 = mySlopeOne.predict("45", "9")
prediction_id1168 = mySlopeOne.predict("45", "1064")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.096444606781006
Prediction for ID:1168 : 5.0


## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [25]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("45", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1064', '1651', '1650', '1645', '1642']


In [26]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1064                Crossfire (1947)
1651    Spanish Prisoner, The (1997)
1650         Butcher Boy, The (1998)
1645         Butcher Boy, The (1998)
1642        Some Mother's Son (1996)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:** Claramente cambian entre el usuario 45 y el usuario 42 (no estoy usando el 457)

In [29]:
# Escribe el nuevo codigo aqui
reclist_slopeone = mySlopeOne.recommend("42", 5)
print('Lista de items según ID:', reclist_slopeone)


Lista de items según ID: ['1589', '1389', '1656', '1064', '1643']


In [30]:
recmovies_slopeone = np.array(reclist_slopeone).astype(int)
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1589    Schizopolis (1996)
1389          Mondo (1996)
1656    Little City (1998)
1064      Crossfire (1947)
1643     Angel Baby (1995)
Name: title, dtype: object

## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:**

## Most Popular

Usamos Most Popular y entrenamos.

In [41]:
# Definicion de objeto "most popular"
most_popular = pyreclab.MostPopular(dataset='u1.base',
                   dlmchar=b'\t',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)

# en teoria no es "entrenar" , objeto "most_popular" solo registra los items más populares en el set de entrenamiento  
most_popular.train()

In [59]:
top_n = 5

recommendList, maprec, ndcg = most_popular.testrec(input_file='u1.test',
                                          dlmchar=b'\t',
                                          header=False,
                                          usercol=0,
                                          itemcol=1,
                                          ratingcol=2,
                                          topn=top_n,
                                          relevance_threshold=2,
                                          includeRated=False)
user_id = 44

ranking = [int(r) for r in most_popular.recommend(str(user_id), top_n, includeRated=False)]
print('Recommendation for user {}: {}'.format(user_id, ranking))

Recommendation for user 44: [50, 100, 181, 286, 1]


In [60]:
recmovies_slopeone = np.array(ranking).astype(int)
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
50                Star Wars (1977)
100                   Fargo (1996)
181      Return of the Jedi (1983)
286    English Patient, The (1996)
1                 Toy Story (1995)
Name: title, dtype: object

## Slope One

In [61]:
reclist_slopeone = mySlopeOne.recommend("44", 5)
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1656', '1064', '1643', '1642', '1625']


In [62]:
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1656          Little City (1998)
1064            Crossfire (1947)
1643           Angel Baby (1995)
1642    Some Mother's Son (1996)
1625           Nightwatch (1997)
Name: title, dtype: object

Tenemos una clarísima diferencia que es muy entendible. El primer método recomienda en base a la popularidad del dataset completo sin ser colaborativo. En cambio en el segundo como evaluamos los ratings del usuario y otros que compartan con este usuario activo, la recomendacion es mucho mas personalizada.